In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [2]:
#df_train = pd.read_csv('./data/raw/train.csv',parse_dates=['click_time'], nrows=1000000)
df_train = pd.read_csv('./data/raw/train.csv',parse_dates=['click_time'])

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184903890 entries, 0 to 184903889
Data columns (total 8 columns):
ip                 int64
app                int64
device             int64
os                 int64
channel            int64
click_time         datetime64[ns]
attributed_time    object
is_attributed      int64
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 11.0+ GB


In [4]:
df_train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0


In [5]:
df_train.drop(['attributed_time'], axis=1, inplace=True)

In [6]:
df_train['hour'] = df_train['click_time'].dt.hour

df_train = df_train.sort_values(['ip', 'click_time'])
df_train['ip_time_diff'] = df_train['click_time'].diff().dt.seconds
df_train.loc[df_train.ip != df_train.ip.shift(), 'ip_time_diff'] = -1

df_train = df_train.sort_values(['ip', 'app','device','os','channel', 'click_time'])
df_train['all_time_diff'] = df_train['click_time'].diff().dt.seconds
df_train.loc[df_train.channel != df_train.channel.shift(), 'all_time_diff'] = -1

In [7]:
df_train.head()

,ip,app,device,os,channel,click_time,is_attributed,hour,ip_time_diff,all_time_diff
180452058,1,1,1,2,125,2017-11-09 14:34:10,0,14,1.0,-1.0
141463813,1,1,1,9,125,2017-11-09 03:01:54,0,3,0.0,44864.0
167268385,1,1,1,48,13,2017-11-09 10:42:11,0,10,465.0,-1.0
125691702,1,2,1,2,477,2017-11-08 17:47:39,0,17,1323.0,-1.0
127330043,1,2,1,2,477,2017-11-08 20:08:39,0,20,177.0,8460.0


In [8]:
df_train['ip_count'] = df_train.groupby('ip')['ip'].transform('count')
df_train['all_count'] = df_train.groupby(['ip', 'app','device','os','channel'])['ip'].transform('count')

In [9]:
df_train.head()

,ip,app,device,os,channel,click_time,is_attributed,hour,ip_time_diff,all_time_diff,ip_count,all_count
180452058,1,1,1,2,125,2017-11-09 14:34:10,0,14,1.0,-1.0,47,1
141463813,1,1,1,9,125,2017-11-09 03:01:54,0,3,0.0,44864.0,47,1
167268385,1,1,1,48,13,2017-11-09 10:42:11,0,10,465.0,-1.0,47,1
125691702,1,2,1,2,477,2017-11-08 17:47:39,0,17,1323.0,-1.0,47,5
127330043,1,2,1,2,477,2017-11-08 20:08:39,0,20,177.0,8460.0,47,5


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184903890 entries, 180452058 to 180096652
Data columns (total 12 columns):
ip               int64
app              int64
device           int64
os               int64
channel          int64
click_time       datetime64[ns]
is_attributed    int64
hour             int64
ip_time_diff     float64
all_time_diff    float64
ip_count         int64
all_count        int64
dtypes: datetime64[ns](1), float64(2), int64(9)
memory usage: 17.9 GB


In [13]:
df_train.to_csv('./data/processed/train.csv', index=False)

In [12]:
df_train.sample(frac=0.4).to_csv('./data/processed/train_sampled.csv', index=False)